### Bibliotecas Utilizadas

In [1]:
#Importar bibliotecas do sistemas
import sys
import os

# Caminho absoluto até a pasta src
sys.path.append(os.path.abspath("../src"))

#Importar libs do python
import pandas as pd 
import numpy as np 
import math
from pyautocad import Autocad, APoint 

#Importar libs do programa
import design_functions
from v_p_cantoneira_flex.v_p_cantoneira_flex  import dim_cant_parafuso
from draw_autocad.draw_autocad_figures import *
from materials import * 

### Dimensionamento da ligação

#### Tipos de Aço escolhidos para o perfil

In [2]:
aço_escolhido = ASTM_A572

aço_escolhido_cantoneira = ASTM_A36

#### Propriedades geométricas do Perfil

In [3]:
perfil_escolhido = W_610x125_0
perfil_escolhido.inercias()
perfil_escolhido.material(aço_escolhido)

#### Propriedades  dos Parafusos

In [4]:
parafuso = ASTM_A325
parafuso.prop_geometricas(rosca = 1,planos_de_corte =2)
diametros = diametros_A325
#Escolha da quantidade de parafusos
n_p_min=1
n_p_max = np.floor((perfil_escolhido.h_w-2*30)/67.5)  # 6.5 é a média do espaçamento entre os parafusos
N_parafusos = 6 

#### Esforços solicitantes e solução

In [5]:
V = 852 #kN

T = 0 #kN


S = dim_cant_parafuso(T,V,cantoneiras_dict,aço_escolhido_cantoneira,perfil_escolhido,parafuso,diametros,N_parafusos,gamma)

In [6]:
cantoneira_escolhida = S[0]
parafuso = S[1]

In [7]:
parafuso.diametro_pol

'5/8'

In [8]:
ver_parafuso = cantoneira_escolhida.disp_parafusos
ver_chapa = cantoneira_escolhida.disp_vertices_chapa

### Criação do desenho

In [9]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Desenho1.dwg


In [10]:
limpar_desenho(acad)

✅ Desenho limpo com sucesso.


#### Gerar coordenadas dos hexágonos

In [11]:
pontos_hexagono = gerar_pontos_hexagono(parafuso.diametro_mm)

#### Desenhar a cantoneira

In [12]:
objetos_s_cantoneira = desenhar_s_cantoneira(acad, cantoneira_escolhida, ver_chapa)

#### Desenhar os parafusos do plano XZ

In [13]:
objetos_p1_cantoneira = []   
 # === Parafusos e hexágonos ===
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 1]
    y_centro = ver_parafuso.iat[i, 2]
    z_centro = ver_parafuso.iat[i, 3]

    # Face do hexágono em X
    obj1 = acad.model.AddCircle(APoint(x_centro, z_centro, -y_centro), parafuso.diametro_mm / 2)
    obj1.Rotate3D(APoint(0, 0, 0), APoint(1, 0, 0), math.radians(90))
    objetos_p1_cantoneira.append(obj1)

    # Face traseira em X
    obj2 = acad.model.AddCircle(APoint(x_centro, z_centro, 0), parafuso.diametro_mm / 2)
    obj2.Rotate3D(APoint(0, 0, 0), APoint(1, 0, 0), math.radians(90))
    objetos_p1_cantoneira.append(obj2)

    # Hexágono desenhado com linhas
    hexagono_transladado = transladar_pontos(pontos_hexagono, x_centro, z_centro, y_centro)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(hexagono_transladado[j][0], hexagono_transladado[j][1], -cantoneira_escolhida.t_mm)
        p2 = APoint(hexagono_transladado[j + 1][0], hexagono_transladado[j + 1][1], -cantoneira_escolhida.t_mm)

        linha = acad.model.AddLine(p1, p2)
        linha.Rotate3D(APoint(0, 0, 0), APoint(1, 0, 0), math.radians(90))
        objetos_p1_cantoneira.append(linha)

#### Desenhar os parafusos do plano XY

In [14]:
objetos_p2_cantoneira = []
# === Parafusos e hexágonos ===
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 2]
    y_centro = ver_parafuso.iat[i, 1]   #Muda a tabela considerando agora os parafusos do outro plano
    z_centro = ver_parafuso.iat[i, 3]

    # Face do hexágono em X
    obj1 = acad.model.AddCircle(APoint(z_centro, y_centro, -x_centro), parafuso.diametro_mm / 2)
    obj1.Rotate3D(APoint(0, 0, 0), APoint(0, 1, 0), math.radians(-90))
    objetos_p2_cantoneira.append(obj1)

    # Face traseira em X
    obj2 = acad.model.AddCircle(APoint(z_centro, y_centro, 0), parafuso.diametro_mm / 2)
    obj2.Rotate3D(APoint(0, 0, 0), APoint(0, 1, 0), math.radians(-90))
    objetos_p2_cantoneira.append(obj2)

    # Hexágono desenhado com linhas
    hexagono_transladado = transladar_pontos(pontos_hexagono, z_centro, y_centro, -y_centro)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(hexagono_transladado[j][0], hexagono_transladado[j][1], -cantoneira_escolhida.t_mm)
        p2 = APoint(hexagono_transladado[j + 1][0], hexagono_transladado[j + 1][1], -cantoneira_escolhida.t_mm)

        linha = acad.model.AddLine(p1, p2)
        linha.Rotate3D(APoint(0, 0, 0), APoint(0, 1, 0), math.radians(-90))
        objetos_p2_cantoneira.append(linha)

#### Desenhar seção das cantoneiras

In [15]:
# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 10, perfil_escolhido.t_w/2, (perfil_escolhido.h-cantoneira_escolhida.comprimento)/2  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

# Aplica a translação a todos os objetos na lista
for obj in objetos_s_cantoneira:
    obj.Move(APoint(0,0,0),vetor) 
    obj.Mirror(APoint(1, 0, 0), APoint(0, 0, 0))
for obj in objetos_p2_cantoneira:
    obj.Move(APoint(0,0,0),vetor) 
    obj.Mirror(APoint(1, 0, 0), APoint(0, 0, 0))
for obj in objetos_p1_cantoneira:
    obj.Move(APoint(0,0,0),vetor) 
    obj.Mirror(APoint(1, 0, 0), APoint(0, 0, 0))

#### Desenhar seção do perfil

In [16]:
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_escolhido, posicao_x=-perfil_escolhido.b_f/2, posicao_y=-perfil_escolhido.h/2, altura_z=0)

# Rotacionar apenas a seção do perfil:
for obj in objetos_secao_perfil:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 0,0,perfil_escolhido.h/2  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

for obj in objetos_secao_perfil:
    obj.Move(APoint(0,0,0),vetor)